In [1]:
import os
import pickle, random
from IPython.display import display, Image
import numpy as np
import faiss
import tqdm
import json, tiktoken
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import difflib, requests
import openai
import random, time


/Users/kamel/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Loading the embedding

We embedded the training and test sets using the OpenAI model. Then, we input these embeddings into FAISS to select the relevant examples for in-context learning.

In [6]:
filel = open('Embedding/PyPIBugs_embedding_Training', 'rb')
PyPiBugs_training = pickle.load(filel)

In [3]:
file = open('Embedding/PyPIBugs_embedding_Testing', 'rb')
PyPiBugs_testing = pickle.load(file)

### FAISS

We implemented FAISS to select the most similar examples from the training set for each test example.

In [5]:
d = PyPiBugs_training.shape[1]
print(d)

1536


In [6]:
index = faiss.IndexFlatL2(d)

In [7]:
index.is_trained

True

In [8]:
index.add(PyPiBugs_training)

In [9]:
index.ntotal

3662

In [10]:
Idx = []
for i in tqdm.tqdm(range(len(PyPiBugs_testing))):
    k = 4
    xq = np.array(PyPiBugs_testing[i][0])
    xq = xq.reshape(1, 1536)
    D, I = index.search(xq, k)
    Idx.append(["Traning_ID", I, "Testing_ID", i])# , "Distance", D

100%|███████████████████████████████████████| 458/458 [00:00<00:00, 1293.49it/s]


In [11]:
Idx[0]

['Traning_ID', array([[1008, 3577, 3576, 1009]]), 'Testing_ID', 0]

### Loading the dataset

In [12]:
with open('/PyPIBugs/train.jsonl', 'r') as json_file:    
    json_list_traning = list(json_file)
    
with open('PyPIBugs/test.jsonl', 'r') as json_file:    
    json_list_testing = list(json_file)

In [13]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [140]:
labeling = []
shot_MAX = 2

### Code-Pair Classification

In [147]:
for counter, index in enumerate(tqdm.tqdm((range(((len(labeling))),len(PyPiBugs_testing), 1)))):
    function_1 = json.loads(json_list_testing[counter])
    system = '<s>[INST]<<SYS>>Can you classify the provided code as <LABEL>buggy</LABEL> or <LABEL>non-buggy</LABEL><</SYS>>'
    prompt = ""
    few_shot = ''
    samples = []
    for index_ in Idx[index][1][0]:
        function = json_list_traning[index_]
        result = json.loads(function)
        func = result['func']
        samples.append(index_)
    
    shot = 0
    for f in samples:

        if shot == shot_MAX:
            break
        shot = + 1
        function = json_list_traning[f]
        result = json.loads(function)
        func = result['func']
        Label = ("buggy") if result["target"] == 0 else ("non-buggy")
        
        few_shot +=  "\nCode: " + func + "\n" + "\n<LABEL>" + Label + '</LABEL>\n'
    
    
    prompt = system + few_shot + str(function_1["func"])
                                      
    response = openai.Completion.create(
        model=model_name,
        prompt=[prompt],
        temperature=0,
        max_tokens=10,
    )
    #time.sleep(13)
    labeling.append(response.choices[0].text)

100%|█████████████████████████████████████████| 165/165 [03:23<00:00,  1.23s/it]


### Print out the results

In [ ]:
Predict_label = []
count = 0
for i in labeling:
    if i[:18] == '\n\n<LABEL>non-buggy':
        count = count + 1
        Predict_label.append(1)
    else:
        Predict_label.append(0)
Correct_label = [0,1] * 229
print(len(Correct_label))
print(len(Predict_label))

In [ ]:
print('f1_score: ', round((f1_score(Correct_label, Predict_label) * 100),2))
print('accuracy_score: ',round((accuracy_score(Correct_label, Predict_label) * 100),2))

In [ ]:
print(classification_report(Correct_label, Predict_label))